In [1]:
from options.test_options import TestOptions
from data import DataLoader
from models import create_model
from models.layers.mesh_prepare import extract_features
from util.writer import Writer
from data.base_dataset import collate_fn
import numpy as np
import random
import torch.nn as nn
import torch
import torch.utils.data

In [2]:
def test_attacked_model(model, dataset, writer):
    writer.reset_counter()
    attacked_model_outputs = []
    
    for i, data in enumerate(dataset):
        model.set_input(data)
        output, ncorrect, nexamples = model.test() 
        attacked_model_outputs.append(output)
        writer.update_counter(ncorrect, nexamples)
        
    writer.print_acc(-1, writer.acc)
    return attacked_model_outputs 
    
def find_new_vertex_index(vertices_edges, edge_index, old_vertex_index):
    for new_vertex_index, new_vertex_edges in enumerate(vertices_edges):
            for new_edge_index in new_vertex_edges:
                if(new_edge_index == edge_index and new_vertex_index != old_vertex_index):
                    return new_vertex_index
                
def get_random_walk(mesh, random_walk_size):
    walk_steps = 0
    random_walk_vertices = []
    random_walk_indices = []
    vertex_index = random.randint(0, len(mesh.vs)-1)
    
    while walk_steps < random_walk_size: 
        random_walk_vertices.append(mesh.vs[vertex_index])
        random_walk_indices.append(vertex_index)        
        walk_steps += 1  
        
        vertex_edges = mesh.ve[vertex_index]
        random_edge_index = random.randint(0, len(vertex_edges)-1)  
        new_edge_index = vertex_edges[random_edge_index]
        vertex_index = find_new_vertex_index(mesh.ve, new_edge_index, random_walk_indices[-1])
        
        #Prevents random walk from crossing over itself
        count_of_vertex_edge_attempts = 0
        walk_steps_backwards = 0
        while(vertex_index in random_walk_indices):
            if(count_of_vertex_edge_attempts >= len(vertex_edges)-1):
                walk_steps_backwards += 1
                go_back_to_index = walk_steps-walk_steps_backwards
                
                if(go_back_to_index == 0):
                    # Trying again, mesh seems to be broken?
                    return get_random_walk(mesh, random_walk_size)
                
                vertex_edges = mesh.ve[random_walk_indices[go_back_to_index]]
                count_of_vertex_edge_attempts = 0
                
            random_edge_index = (random_edge_index + 1) % len(vertex_edges)
            new_edge_index = vertex_edges[random_edge_index]
            vertex_index = find_new_vertex_index(mesh.ve, new_edge_index, random_walk_indices[-1])
            count_of_vertex_edge_attempts += 1        
    
    return torch.FloatTensor(random_walk_vertices), random_walk_indices

In [3]:
def train_imitating_network(imitating_nn, criterion, optimizer, random_walk_vertices, attacked_nn_output):

    output = imitating_nn(random_walk_vertices)
    loss = criterion(output, attacked_nn_output)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step() 

    return output, loss.item()

In [4]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)

    def forward(self, input, hidden):
        
        combined = torch.cat((input, hidden), -1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

class Imitating_NN(nn.Module):
    def __init__(self, input_size, output_size):
        super(Imitating_NN, self).__init__()
        
        self.num_vertices = 3
        self.scaling_factor = 10
        
        self.first_linear = nn.Linear(input_size, input_size*self.scaling_factor)
        self.second_linear = nn.Linear(input_size*self.scaling_factor, input_size*self.scaling_factor)
        self.rnn = RNN(self.scaling_factor*input_size, input_size, self.scaling_factor*input_size)
        self.third_linear = nn.Linear(self.scaling_factor*input_size, output_size)
        
    def forward(self, random_walk_vertices):
        
        transformed_random_walk = self.first_linear(random_walk_vertices)
        transformed_random_walk = self.second_linear(transformed_random_walk)
        
        hidden = self.rnn.initHidden()
        for step in transformed_random_walk:
            output, hidden = self.rnn(torch.reshape(step, (1, self.num_vertices*self.scaling_factor)), hidden)
            
        output = self.second_linear(output)
            
        return output
        

# Orchestration

In [5]:
testing_opt = TestOptions().parse()

testing_opt.serial_batches = True
dataloader = DataLoader(testing_opt)
mesh_cnn = create_model(testing_opt)
opt_writer = Writer(testing_opt)

random_walk_size = 50
num_categories = 30
num_vertice_coordinates = 3
imitating_nn = Imitating_NN(num_vertice_coordinates, num_categories)
criterion = nn.KLDivLoss(reduction="batchmean")
optimizer = torch.optim.Adam(imitating_nn.parameters(), lr = 0.0001)

loaded mean / std from cache
loading the model from ./checkpoints/shrec16/latest_net.pth


In [6]:
# Record testing accuracy before attack
attacked_model_outputs = test_attacked_model(mesh_cnn, dataloader, opt_writer)

epoch: -1, TEST ACC: [99.167 %]



In [7]:
# Random Walk Attack
for epoch in range(10):
    epoch_loss = 0
    
    for i, data in enumerate(dataloader):
        print("i: " + str(i))
        print("Mesh labels: " + str(data["label"]))
    
        mesh = data["mesh"][0]
        random_walk_vertices, random_walk_indices = get_random_walk(mesh, random_walk_size)
    
        imitating_nn_output, loss = train_imitating_network(imitating_nn, criterion, optimizer, random_walk_vertices, attacked_model_outputs[i])
        epoch_loss += loss
    
        print(attacked_model_outputs[i])
        print(imitating_nn_output)
    
    print("epoch: " + str(epoch) + ", loss: " + str(epoch_loss))
    

i: 0
Mesh labels: [0]
tensor([[  5.4885, -19.8533, -21.4682,  -9.6839,  -8.4958,  -3.6247,  -4.4009,
          -6.7783, -23.9704, -19.1216, -15.8959,  -6.0566,  -4.8584, -11.2263,
         -16.3660, -13.7719,  -8.6655, -10.7375,  -5.2502, -18.5240,  -0.8722,
         -19.7822, -22.9073,  -3.3643, -25.6530,  -7.8275, -25.9369, -18.4495,
          -3.2837, -13.6820]])
tensor([[ 0.1982, -0.0295,  0.0188,  0.0736,  0.0445,  0.0668, -0.0656, -0.0698,
          0.0070,  0.1290, -0.0998,  0.0623,  0.0270,  0.0138, -0.2009, -0.0837,
          0.0525,  0.1806, -0.0441, -0.1633, -0.1165,  0.1817, -0.0232,  0.0348,
          0.0736, -0.0719,  0.1241,  0.1242, -0.0642,  0.2423]],
       grad_fn=<AddmmBackward0>)
i: 1
Mesh labels: [0]
tensor([[  6.3456, -20.1829, -23.7263, -11.1970, -10.4479,  -3.5765,  -4.1582,
          -5.3284, -26.1639, -20.5226, -15.8104,  -4.7861,  -4.6728, -11.7994,
         -17.7693, -14.5876,  -7.8231, -13.3061,  -7.3532, -18.4909,  -1.7529,
         -21.3954, -25.2309,  -

/var/folders/ts/sy6vbbcs1vdbgtnds0713g6w0000gq/T/ipykernel_10221/1098266402.py:56: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:204.)
  return torch.FloatTensor(random_walk_vertices), random_walk_indices


i: 14
Mesh labels: [3]
tensor([[-13.9660, -12.2713, -12.0443,   5.6351,  -3.6740,  -6.9553,  -4.5959,
         -17.9461, -17.6826,  -5.2133, -19.0052, -23.7066, -23.1402,  -9.8901,
         -10.1013,  -2.3390, -25.3224, -11.6810,  -3.6288, -20.4719, -12.9614,
          -5.8187, -22.8622,  -8.5795, -23.6955, -13.3311, -20.4259, -11.6520,
         -14.3199, -25.3500]])
tensor([[ 0.2968, -0.0773,  0.0299,  0.0698,  0.0230,  0.0045, -0.0818, -0.0227,
          0.0665,  0.0972, -0.0511,  0.0038,  0.0812,  0.0617, -0.1931, -0.1628,
          0.0725,  0.2657, -0.0692, -0.2055, -0.1342,  0.1725, -0.0159,  0.0047,
          0.1616, -0.0826,  0.1403,  0.1419, -0.0291,  0.2535]],
       grad_fn=<AddmmBackward0>)
i: 15
Mesh labels: [3]
tensor([[-14.2403, -12.7981, -11.2377,   5.2370,  -3.5653,  -7.9844,  -4.7698,
         -18.8413, -16.3065,  -5.1471, -18.1189, -23.2033, -22.8174, -10.6906,
          -8.7109,  -2.6504, -25.2361, -11.8052,  -3.2504, -19.7953, -12.6823,
          -7.1463, -21.5870, 

tensor([[ -9.8054,  -3.0362,  -7.7672, -16.0421, -22.0341, -12.0270,  -2.7380,
          -6.8935, -11.8634, -10.3999,   5.5222,  -3.5391, -19.0065, -24.9962,
          -3.9015,  -6.5952,  -1.8166, -26.7677, -17.2053,  -4.8793, -22.5182,
         -27.6809, -20.6622,  -4.9033,  -3.8696, -14.3837, -25.3255, -18.7351,
          -3.4505,  -5.7859]])
tensor([[ 0.2675, -0.0421,  0.0488,  0.0796,  0.0361,  0.0249, -0.0872, -0.0417,
          0.0557,  0.1098, -0.0800,  0.0358,  0.0596,  0.0449, -0.2242, -0.1418,
          0.0629,  0.2482, -0.0370, -0.1995, -0.1167,  0.1754, -0.0257,  0.0304,
          0.1314, -0.0587,  0.1321,  0.1506, -0.0372,  0.2499]],
       grad_fn=<AddmmBackward0>)
i: 41
Mesh labels: [10]
tensor([[-11.2173,  -6.1620, -11.1898, -17.2096, -22.5225, -13.7300,  -3.1268,
          -8.5890, -14.1684, -11.9777,   3.7165,  -3.9115, -19.1191, -25.7827,
          -3.5140,  -8.1554,  -4.8522, -29.2217, -18.6284,  -5.1977, -23.2068,
         -30.7806, -21.5459,  -7.0243,  -5.5154, -1

tensor([[-15.8641, -16.7311, -30.2572, -11.1232,  -7.8833,  -6.1485,  -5.6824,
          -9.0799, -18.5692,  -5.9895, -27.9221, -15.6250,  -9.1639,   6.2900,
         -16.3776, -13.1409, -25.0122, -17.8039, -19.6985,  -6.8510, -11.1158,
          -6.7717, -12.2290, -21.9364, -25.7201,  -6.3351,  -2.2219,  -9.3106,
         -20.5732, -13.2399]])
tensor([[ 0.2910, -0.0547,  0.0431,  0.0878,  0.0240,  0.0217, -0.0808, -0.0311,
          0.0694,  0.1189, -0.0352,  0.0290,  0.0769,  0.0513, -0.1855, -0.1440,
          0.0724,  0.2470, -0.0813, -0.2085, -0.1243,  0.1756, -0.0155,  0.0147,
          0.1549, -0.0927,  0.1337,  0.1235, -0.0300,  0.2485]],
       grad_fn=<AddmmBackward0>)
i: 53
Mesh labels: [13]
tensor([[-12.7686, -17.5714, -30.0671, -10.8246,  -6.6834,  -4.9487,  -5.6762,
          -8.2139, -19.3342,  -7.7499, -27.4529, -14.2749,  -6.1452,   5.8931,
         -16.9944, -13.9010, -22.8718, -15.9904, -17.8295,  -8.2325,  -8.0531,
          -7.1127, -12.5369, -20.3526, -26.0155,  -

...

tensor([[-13.8156, -12.8400, -23.8924,  -9.2676,  -7.2072,  -5.5136,  -4.7157,
          -7.4916, -14.0714,  -3.9144, -22.4206, -13.1019,  -8.7493,   5.0935,
         -12.8874, -10.2377, -20.8854, -15.0960, -16.5194,  -4.8228, -10.2136,
          -5.1903,  -9.4925, -18.2005, -20.1399,  -5.3534,  -0.6626,  -6.9383,
         -17.4111, -10.8901]])
tensor([[ 1.5809,  0.8590,  1.8392,  0.5070,  0.3053,  0.5762, -0.2666,  0.4997,
          1.7751,  0.8305,  1.3500,  0.9155,  1.2937,  0.7356,  0.1438, -0.3184,
          1.1155,  2.2883,  1.3316, -0.5428,  0.3988,  0.9145,  0.4494,  0.6314,
          1.6038,  0.8572,  0.9281,  1.1318,  0.8361,  0.9565]],
       grad_fn=<AddmmBackward0>)
i: 55
Mesh labels: [13]
tensor([[-14.7091, -18.4965, -29.1599, -10.0573,  -4.9917,  -5.9170,  -5.3289,
         -10.4114, -17.2106,  -6.2029, -27.3137, -14.5589,  -6.6187,   6.1483,
         -15.1838, -13.4674, -24.3929, -15.6845, -17.5054,  -7.2159,  -8.2009,
          -7.0815, -10.0279, -21.1741, -24.5894,  -

tensor([[ -6.8667, -15.2529,  -4.2042,   0.6357,  -4.3796,  -7.5659,  -6.3964,
         -18.6289, -16.0172, -10.4212, -15.2902, -20.4248, -20.2556, -16.4315,
          -7.5203,  -4.4047, -18.0207,  -2.8373,   7.9993, -23.6170,  -9.4781,
         -11.8062, -22.7356,   0.1718, -19.7938, -12.7229, -22.2121, -16.9140,
          -3.4312, -21.4502]])
tensor([[ 1.4331,  0.8932,  1.7413,  0.5133,  0.2952,  0.6310, -0.2613,  0.3729,
          1.6282,  0.8666,  1.2664,  1.0105,  1.2345,  0.7123,  0.1382, -0.1850,
          1.0941,  2.1405,  1.2634, -0.5241,  0.3910,  0.8990,  0.4218,  0.6510,
          1.4389,  0.8088,  0.8786,  1.0423,  0.7397,  0.9110]],
       grad_fn=<AddmmBackward0>)
i: 75
Mesh labels: [18]
tensor([[ -6.2519, -15.5311,  -4.6246,   0.0701,  -3.5657,  -6.7552,  -6.3899,
         -18.1952, -15.8199, -10.5677, -15.7134, -19.7100, -18.7206, -15.6698,
          -8.1301,  -5.1543, -17.3226,  -1.5431,   8.1775, -23.3953,  -8.3856,
         -11.3226, -21.4789,   0.1169, -19.7186, -1

tensor([[-17.1850,  -7.1073, -14.7938,  -1.7395,  -2.3789,   1.4221,  -6.6295,
         -13.9747, -20.0892,  -2.9650, -25.4700, -23.6302, -17.1402,  -1.8111,
         -17.4176,  -3.7117, -21.3280,  -0.1791,  -7.0395, -18.3350, -16.8462,
           9.6003, -18.4236, -11.0242, -24.9167, -19.4055, -13.2250,  -1.3964,
         -19.1988, -27.6331]])
tensor([[ 1.6419,  0.9456,  1.9901,  0.5458,  0.3282,  0.6878, -0.3218,  0.5054,
          1.9116,  0.9305,  1.4745,  1.0765,  1.4451,  0.8439,  0.1706, -0.2438,
          1.2924,  2.5480,  1.5530, -0.5548,  0.5040,  0.9641,  0.4882,  0.6881,
          1.6754,  0.9707,  0.9933,  1.1959,  0.8686,  1.0211]],
       grad_fn=<AddmmBackward0>)
i: 88
Mesh labels: [22]
tensor([[-24.1869, -16.2579, -18.5360, -14.4957,  -6.7891, -15.6602,  -7.5203,
         -21.3831,  -1.1955,  -1.4690, -14.5474, -12.5893, -12.0231,  -8.8962,
          -3.5067, -15.8422, -23.6550, -18.1094, -19.6146,   0.5762, -14.0230,
         -19.8573,   9.2143, -25.6817,  -3.6654,  -

tensor([[-11.3650, -25.5086, -23.8257,  -9.8779,  -2.8680, -14.9700,  -4.6951,
         -18.6867,  -7.7704,  -8.8725, -18.9919,  -9.7815,  -5.2168,  -4.0773,
          -7.1178, -16.8865, -23.2980, -19.4247, -13.4267,  -7.0261,  -1.6310,
         -22.3703,  -4.7941, -19.6495, -16.0471,   7.0702,  -4.1602, -28.6626,
         -10.3997,  -2.9798]])
tensor([[ 1.6818,  0.9390,  2.0181,  0.5443,  0.3458,  0.7279, -0.2829,  0.5516,
          1.9606,  0.9260,  1.5087,  1.0474,  1.4659,  0.8393,  0.2599, -0.2348,
          1.3164,  2.6045,  1.6041, -0.5272,  0.5464,  1.0390,  0.5636,  0.7122,
          1.7637,  0.9667,  0.9883,  1.2121,  0.9130,  1.0155]],
       grad_fn=<AddmmBackward0>)
i: 103
Mesh labels: [25]
tensor([[-14.3204, -22.8477, -19.3216,  -8.9437,  -4.2516, -16.6635,  -5.2487,
         -20.2665,  -4.2940,  -5.9561, -17.7130, -13.0189, -10.2812,  -5.3670,
          -5.5154, -14.8676, -24.8720, -20.1338, -12.8372,  -6.9348,  -4.8901,
         -21.1873,  -4.6956, -19.3544, -13.7973,  

tensor([[-14.9681, -21.4830, -27.5907, -20.2782, -12.7485, -15.4414,  -4.5263,
         -14.4074, -14.7664, -13.7452,  -9.4865,  -0.9243,  -3.6304, -15.9467,
          -3.5095, -19.6070, -15.4743, -26.7960, -21.8208,   0.1929, -12.7745,
         -33.8928,  -6.2963, -21.3034,  -7.6566,  -2.9492, -13.2439, -34.3099,
          -4.4751,   5.7164]])
tensor([[ 1.6898,  1.0193,  2.0805,  0.5355,  0.3747,  0.7732, -0.2685,  0.5569,
          2.0199,  0.9597,  1.5404,  1.0991,  1.4886,  0.8497,  0.2816, -0.2156,
          1.3316,  2.6513,  1.6907, -0.5241,  0.5726,  1.0811,  0.5862,  0.7709,
          1.8183,  1.0756,  1.0828,  1.3111,  0.9537,  1.0367]],
       grad_fn=<AddmmBackward0>)
i: 117
Mesh labels: [29]
tensor([[-13.3109, -17.6586, -22.6130, -18.1277, -13.3338, -13.9144,  -5.0583,
         -12.6665, -15.2249, -13.3974,  -7.8443,  -2.6484,  -5.6877, -16.2637,
          -2.4342, -15.8707, -13.2478, -23.3710, -17.6797,  -1.9487, -13.5214,
         -30.4678,  -9.6489, -16.5979,  -7.3042,  

In [10]:
# Random Pertubation Attack
dataloader = DataLoader(testing_opt)
overridden_meshes = []
for i, data in enumerate(dataloader):
    
    mesh = data["mesh"][0]
    num_random_changes = 0
    
    while(num_random_changes < 10):
        
        random_vertex_index = random.randint(0, len(mesh.vs)-1)  
        random_coordinate_index = random.randint(0, 2)  
        mesh.vs[random_vertex_index][random_coordinate_index] += random.uniform(-0.2, 0.2)
        mesh.features = extract_features(mesh)
        num_random_changes += 1
    
    overridden_meshes.append(mesh)

dataloader.dataloader.dataset.override_meshes(overridden_meshes) 

loaded mean / std from cache


In [11]:
# Record testing accuracy after attack
attacked_model_outputs = test_attacked_model(mesh_cnn, dataloader, opt_writer)

epoch: -1, TEST ACC: [95.0 %]

